In [1]:
from tensorflow.keras.applications import *
from backbone.mobilenet_v3  import MobileNetV3Small as  MobileNetV3Small_M

import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"]="3"

model = MobileNetV3Small(input_shape=(112,112,3), include_top=False, weights='imagenet')

In [2]:
model2 = MobileNetV3Small_M(input_shape=(112,112,3), include_top=False, weights=None)

In [3]:
model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 112, 112, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 56, 56, 16)   432         rescaling[0][0]                  
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 56, 56, 16)   64          Conv[0][0]                       
___________________________________________________________________________________

In [4]:
model2.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
Conv (Conv2D)                   (None, 56, 56, 16)   432         input_2[0][0]                    
__________________________________________________________________________________________________
Conv/BatchNorm (BatchNormalizat (None, 56, 56, 16)   64          Conv[0][0]                       
__________________________________________________________________________________________________
tf.__operators__.add_28 (TFOpLa (None, 56, 56, 16)   0           Conv/BatchNorm[0][0]             
___________________________________________________________________________________

__________________________________________________________________________________________________
re_lu_49 (ReLU)                 (None, 7, 7, 120)    0           tf.__operators__.add_39[0][0]    
__________________________________________________________________________________________________
tf.math.multiply_39 (TFOpLambda (None, 7, 7, 120)    0           re_lu_49[0][0]                   
__________________________________________________________________________________________________
multiply_26 (Multiply)          (None, 7, 7, 120)    0           tf.math.multiply_39[0][0]        
                                                                 expanded_conv_6/expand/BatchNorm[
__________________________________________________________________________________________________
expanded_conv_6/depthwise (Dept (None, 7, 7, 120)    3000        multiply_26[0][0]                
__________________________________________________________________________________________________
expanded_c

EfficientNet의 경우 stem에서 width scaling을 하는데 그로 인해 1번째 block1a에서 weight를 불러올수 없었다
EfficientNetLite의 경우 이렇게 불러오면 불완전하지만 [official code](https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/lite/README.md)를 사용하는 방법을 터득하기전까지는 이렇게 사용하자

In [5]:
# EfficientNet Lite
# for layer in model2.layers:
#     if (not ("input_" in layer.name)):
#         if (not ("stem_" in layer.name)):
#             if (not ("block1a_" in layer.name)):
#                 if (not ("top_" in layer.name)):
#                     print(layer.name)
#                     model2.get_layer(layer.name).set_weights(model.get_layer(layer.name).get_weights())

# MobileNet
for layer in model2.layers:
    if (not ("input_" in layer.name)):
        if (not ("add_" in layer.name)):
            if (not ("multiply_" in layer.name)):
                if (not ("re_lu_" in layer.name)):
                    if (not ("reshape_" in layer.name)):
                        model2.get_layer(layer.name).set_weights(model.get_layer(layer.name).get_weights())

In [6]:
# model2.save_weights('./weights/efficientnet_lite'+version+'_notop.ckpt')
model2.save_weights('./weights/mobilenet_v3_small_notop.ckpt')